In [21]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset("SetFit/subj")
dataset = dataset.remove_columns("label_text")
dataset = dataset['train'].shuffle(seed=42).select([i for i in list(range(6000))])
train_ds = Dataset.from_dict(dataset[0:5000])
validation_ds = Dataset.from_dict(dataset[5000:5500])
test_ds = Dataset.from_dict(dataset[5500:6000])



display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)



Using custom data configuration SetFit___subj-848ec40cb3887b72
Reusing dataset json (C:\Users\DELL\.cache\huggingface\datasets\json\SetFit___subj-848ec40cb3887b72\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)
100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.46it/s]
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\json\SetFit___subj-848ec40cb3887b72\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-80b8e4129e527f71.arrow


Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

,text,label
0,"on this morning , johnny wakes to his mother '...",0
1,"despite a quieter middle section , involving a...",1
2,"[ kidd ] can write dialogue , work skillfully ...",1
3,"abandoned and filled with a mysterious past , ...",0
4,the romantic triangle leads to a surprising co...,0
...,...,...
4995,"jacobi , the most fluent of actors , is given ...",1
4996,even if you do n't understand what on earth is...,1
4997,"gerry 's daughter , mattie , accidentally prin...",0
4998,a well paced and satisfying little drama that ...,1


In [22]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [23]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore

from textattack.augmentation import Augmenter
from textattack.augmentation.recipes import CheckListAugmenter





# Create augmenter with specified parameters
augmenter = CheckListAugmenter(pct_words_to_swap=0.1, transformations_per_example=1)

In [24]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):
    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)



time_end = time.perf_counter()
tim = time_end - time_sta

In [25]:


print(tim/3600)




0.0926091227222222


In [26]:
display(results_list)

[["on this morning , johnny wakes to his mother 's accusations that he has botched a simple scam , leaving them without enough money to payoff their inside man ."],
 ["despite a quieter middle section , involving aragorn 's dreams of arwen , this is even better than the fellowship . there are scenes of cinematic perfection that steal your heart away ."],
 ['[ kidd ] can write dialogue , work skillfully with actors , and he has a pretty good handle on urban loneliness of the knowing , virulent new york city variety .'],
 ['abandoned and filled with a mysterious past , holy angel school for girls is about to reveal its secrets of betrayal , jealousy , and vengeance .'],
 ["the romantic triangle leads to a surprising conclusion as the young man 's secret motive has explosive consequences ."],
 ['as a film director , labute continues to improve .'],
 ['now , along with other surviors , jill valentine , Maureen Cummins and carlos oliviera , they must fight to survive , to escape the nightma

In [27]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


[["on this morning , johnny wakes to his mother 's accusations that he has botched a simple scam , leaving them without enough money to payoff their inside man ."],
 ["despite a quieter middle section , involving aragorn 's dreams of arwen , this is even better than the fellowship . there are scenes of cinematic perfection that steal your heart away ."],
 ['[ kidd ] can write dialogue , work skillfully with actors , and he has a pretty good handle on urban loneliness of the knowing , virulent new york city variety .'],
 ['abandoned and filled with a mysterious past , holy angel school for girls is about to reveal its secrets of betrayal , jealousy , and vengeance .'],
 ["the romantic triangle leads to a surprising conclusion as the young man 's secret motive has explosive consequences ."],
 ['as a film director , labute continues to improve .'],
 ['now , along with other surviors , jill valentine , claire redfield and carlos oliviera , they must fight to survive , to escape the nightma

In [28]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


,original_text,perturbed_text
0,"on this morning , johnny wakes to his mother '...","on this morning , johnny wakes to his mother '..."
1,"despite a quieter middle section , involving a...","despite a quieter middle section , involving a..."
2,"[ kidd ] can write dialogue , work skillfully ...","[ kidd ] can write dialogue , work skillfully ..."
3,"abandoned and filled with a mysterious past , ...","abandoned and filled with a mysterious past , ..."
4,the romantic triangle leads to a surprising co...,the romantic triangle leads to a surprising co...
...,...,...
4995,"jacobi , the most fluent of actors , is given ...","Anneliese , the most fluent of actors , is giv..."
4996,even if you do n't understand what on earth is...,even if you do n't understand what on earth is...
4997,"gerry 's daughter , mattie , accidentally prin...","gerry 's daughter , mattie , accidentally prin..."
4998,a well paced and satisfying little drama that ...,a well paced and satisfying little drama that ...


In [29]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

,original_text,perturbed_text
0,"now , along with other surviors , jill valenti...","now , along with other surviors , jill valenti..."
1,nolan bravely treads where few american films ...,Kelia bravely treads where few american films ...
2,"leela is the story of leela , an aware , liber...","Ashli is the story of leela , an aware , liber..."
3,but even this new relationship is not enough t...,but even this new relationship isn't enough to...
4,nachtwey clears the cynicism right out of you ...,Jessica clears the cynicism right out of you ....
...,...,...
1950,john has become not just a full fledged phenom...,Atziri has become not just a full fledged phen...
1951,"join edgar , the man , and gigi , the dog , on...","join Tavin , the man , and gigi , the dog , on..."
1952,"facing her past , culturally and emotionally ,...","facing her past , culturally and emotionally ,..."
1953,it is no coincidence that funding for `` anti-...,it's no coincidence that funding for `` anti-g...


In [30]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\augmented_datasets\CheckList\run3\dataset.csv')